<a href="https://colab.research.google.com/github/madarasw/Advanced_deep_learning_module/blob/main/248084L.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import csv
import math
import numpy as np
import pandas as pd

In [ ]:
#TODO: Define the Neurone class
# remember: np.float32

class Neuron:
    def __init__(self, activation):

        # Parameters
        self.weights = np.empty(1, dtype=np.float32)
        self.bias = 0
        self.activation = activation # Activation function

        # Inputs and Outputs
        self.inputs = np.empty(1, dtype=np.float32)
        self.a = np.float32(1.0)
        self.z = np.float32(1.0)

        # Derivatives
        self.dz_dw = []
        self.dz_db = 1

    # Activation functions
    def relu(self, x):
        return x if x > 0 else 0

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def set_weights(self, weights):
        self.weights = weights

    def set_bias(self, bias):
        self.bias = bias

    # Forward pass
    def forward_prop(self, inputs):
        self.inputs = np.array(inputs, dtype=np.float32)
        self.a = np.dot(self.weights, self.inputs) + self.bias  # weighted sum + bias
        if self.activation == 'ReLU':
            self.z = self.relu(self.a)
        elif self.activation == 'Sigmoid':
            self.z = self.sigmoid(self.a)
        else:
            self.z = self.a  # linear

        return self.z

    def get_activation_gradient(self):
        dz_da = 1
        if self.activation == "ReLU":
          if self.z <= 0:
            dz_da = 0
          else:
            dz_da = 1
        elif self.activation == "Sigmoid":
          dz_da = self.z * (1 - self.z)
        else:
          dz_da = 1                          #for linear activation

        return dz_da

    # backward pass
    def backward_pass(self):

        dz_da = self.get_activation_gradient()  #Scalaer value
        da_dw = self.inputs                     #vector - eg: [z1,z2,z3]

        # gradients of weights
        self.dz_dw = []
        for dadw in da_dw:
            self.dz_dw.append(dz_da*dadw)       #vector

        # gradients of bias
        da_db = 1
        self.dz_db = dz_da*da_db                #Scalar value

        # d_output_by_d_input
        da_dinput = self.weights                #vector - eg: [w1,w2,w3]
        dz_din = []
        for dadinput in da_dinput:
            dz_din.append(dz_da*dadinput)
        return dz_din                           #vector

    def get_w_grad(self):
        return self.dz_dw

    def get_b_grad(self):
        return self.dz_db

In [ ]:


'''
Testing Neuron class
====================

weights = [2,-1,6]
bias = 8
activation = 'relu'

my_neu = Neuron(activation)

my_neu.set_weights(weights)
my_neu.set_bias(bias)

my_neu.forward_prop([2,-1,3])
my_neu.backward_pass(4)

w = my_neu.get_w_grad()
b = my_neu.get_b_grad()

#print(w)
#print(b)
'''


"\nTesting Neuron class\n====================\n\nweights = [2,-1,6]\nbias = 8\nactivation = 'relu'\n\nmy_neu = Neuron(activation)\n\nmy_neu.set_weights(weights)\nmy_neu.set_bias(bias)\n\nmy_neu.forward_prop([2,-1,3])\nmy_neu.backward_pass(4)\n\nw = my_neu.get_w_grad()\nb = my_neu.get_b_grad()\n\n#print(w)\n#print(b)\n"

In [ ]:
#TODO: Write the Layer class

class Layer:
    def __init__(self, Label, N, activation):

        # parameters
        self.label = Label
        self.N = N # layer size (Number of neurons in the layer)
        self.activation = activation
        self.global_derivative = 1

        # neurons
        self.neurons = []
        self.add_neurons()

        # layer output
        self.z = []

    def add_neurons(self):
        for n in range(self.N):
          new_neuron = Neuron(self.activation)
          self.neurons.append(new_neuron)

    def set_weights(self, weights):
        for n in range(self.N):
          w = weights[:, n:n+1]
          weights_ = [item for sublist in w for item in sublist] # remove sublists
          self.neurons[n].set_weights(weights_)

    def set_bias(self, bias):
        for n in range(self.N):
          self.neurons[n].set_bias(bias[n])

    def forward_prop(self, inputs):
        for neuron in self.neurons:
            self.z.append(neuron.forward_prop(inputs))

        return self.z

    def backward_pass(self, global_derivative):
        local_derivative = []
        self.global_derivative = global_derivative
        for n in range(self.N):
            gd = self.neurons[n].backward_pass()  #vector
            local_derivative.append(gd)           #matrix

        dj_dz_lower = np.matmul(self.global_derivative, local_derivative)
        return dj_dz_lower

    def get_w_gradients(self):
        dj_dw = []
        for n in range(len(self.neurons)):
            dz_dw_single = self.neurons[n].get_w_grad() #vector
            dj_dw_single = []
            for element in dz_dw_single:
                dj_dw_single.append(self.global_derivative[n] * element)
            dj_dw.append(dj_dw_single) #matrix

        return dj_dw

    def get_b_gradients(self):
        dj_db = []
        for n in range(len(self.neurons)):
            local_derivative = self.neurons[n].get_b_grad()
            dj_db.append(self.global_derivative[n] * local_derivative)

        return dj_db

##########

In [ ]:
'''
Testing Layer class
====================

l = 0
layer_sizes = [3,2,1]
layer_activations = ['ReLU', 'ReLU', 'ReLU', 'Sigmoid']
weights = np.array([[2,6,2],[1,1,8],[4,2,6]], dtype=np.float32)
bias = np.array([1,8,4], dtype=np.float32)

layer = Layer(f"Layer_{l+1}", layer_sizes[l], layer_activations[l])
layer.set_weights(weights)
layer.set_bias(bias)
layer.forward_prop([2,-1,3])
# layer.backward_pass(4)
'''

'\nTesting Layer class\n====================\n\nl = 0\nlayer_sizes = [3,2,1]\nlayer_activations = [\'ReLU\', \'ReLU\', \'ReLU\', \'Sigmoid\']\nweights = np.array([[2,6,2],[1,1,8],[4,2,6]], dtype=np.float32)\nbias = np.array([1,8,4], dtype=np.float32)\n\nlayer = Layer(f"Layer_{l+1}", layer_sizes[l], layer_activations[l])\nlayer.set_weights(weights)\nlayer.set_bias(bias)\nlayer.forward_prop([2,-1,3])\n# layer.backward_pass(4)\n'

In [ ]:
# TODO: Define the NeuralNetwork Class

class neuralNetwork:

    def __init__(self, network_depth, layer_sizes, layer_activations):
        self.depth = network_depth
        self.layer_sizes = layer_sizes
        self.layer_activations = layer_activations
        self.layers = []
        self.create_layers()
        self.loss = 0


    def create_layers(self):
        for l in range(self.depth):
          # set random values for initial parameters
          layer = Layer(f"Layer_{l+1}", self.layer_sizes[l], self.layer_activations[l])
          self.layers.append(layer)

    def set_weights(self, weight_vector):
        for l in range(self.depth):
            self.layers[l].set_weights(weight_vector[l])

    def set_bias(self, bias_vector):
        for l in range(self.depth):
            self.layers[l].set_bias(bias_vector[l])

    def forward_prop(self, input):
        for layer in self.layers:
            layer_output = layer.forward_prop(input)
            input = layer_output

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_and_binary_cross_entropy_derivative(self, y_true, y_pred):
        # Derivative of Sigmoid and Cross entropy loss together
        dJ_dz = y_pred - y_true
        return dJ_dz


    def backward_prop(self, y_label):
        # Calculate dj_dz
        last_layer_output = self.layers[-1].z
        sigmoid_output = self.sigmoid(last_layer_output[0])

        #dJ_da_final
        global_derivative = [self.sigmoid_and_binary_cross_entropy_derivative(y_label, sigmoid_output)] # dj_dz_final

        #backpropagation
        for layer in reversed(self.layers):
            global_derivative = layer.backward_pass(global_derivative)


    def get_w_gradients(self):
        w_gradients = []
        for layer in self.layers:
            w_gradients.append(layer.get_w_gradients())
        return w_gradients

    def get_b_gradients(self):
        b_gradients = []
        for layer in self.layers:
            b_gradients.append(layer.get_b_gradients())
        return b_gradients

# ------------------------

In [ ]:
# Input Processing

def input_processing(index):
  processed = np.array([d for d in str(index)], dtype=np.float32)
  processed = processed / 9.0
  return processed

def read_weights():
  file_path = '/content/drive/My Drive/Colab Notebooks/Advanced_Deep_Learning/W.csv'
  weights = pd.read_csv(file_path, header=None, float_precision='round_trip')
  weights.drop(weights.columns[[0]], axis=1, inplace=True)
  weights = np.array(weights, dtype=np.float32)
  return weights

def read_bias():
  file_path = '/content/drive/My Drive/Colab Notebooks/Advanced_Deep_Learning/b.csv'
  bias = pd.read_csv(file_path, header=None, float_precision='round_trip')
  bias.drop(bias.columns[[0]], axis=1, inplace=True)
  bias = np.array(bias, dtype=np.float32)
  return bias

In [ ]:

# Read weights and biases
weights = read_weights()
biases = read_bias()

# Input and expected output preprocessing
index = 248084
input = input_processing(index)
y_label = 0 #Index number 248084 is not a prime number

# Specify the network properties
network_depth = 4 # excluding input layer
layer_sizes = [64,32,16,1]

layer_activations = ['ReLU', 'ReLU', 'ReLU', 'Linear'] # set last layer activation as linear to get the logit

# assign values to the weight vector and bias vector
weight_vector = []
bias_vector = []

weight_vector.append(weights[0:6,:64])
weight_vector.append(weights[6:70,:32])
weight_vector.append(weights[70:102,:16])
weight_vector.append(weights[102:118,:1])

for i in range(network_depth):
    bias_vector.append(biases[i, :layer_sizes[i]])

In [ ]:
# Create the neural network using the neuralNetwork class
primeNeuralNetwork = neuralNetwork(network_depth, layer_sizes, layer_activations)

# Set weights and biases of the neural network
primeNeuralNetwork.set_weights(weight_vector)
primeNeuralNetwork.set_bias(bias_vector)

# Forward propagation using the input
primeNeuralNetwork.forward_prop(input)
# Backword propagation using the y label for the given input
primeNeuralNetwork.backward_prop(y_label)

# Get gradients for weights and bias for each neuron
w_gradients = primeNeuralNetwork.get_w_gradients()
b_gradients = primeNeuralNetwork.get_b_gradients()

#print('---------------------------- FINAL RESULT ----------------------------')
#print(w_gradients)
#print(b_gradients)

# Write output in to csv files
csv_filename = '/content/drive/My Drive/Colab Notebooks/Advanced_Deep_Learning/dw_just.csv'
with open(csv_filename, 'w', newline='') as file:
    writer = csv.writer(file)
    for g in w_gradients:
      transposed = np.array(g).T
      writer.writerows(transposed)

csv_filename = '/content/drive/My Drive/Colab Notebooks/Advanced_Deep_Learning/db?_just.csv'
with open(csv_filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(b_gradients)

In [ ]:
'''
TESTING
=======================
'''

def read_weights():
  file_path = '/content/drive/My Drive/Colab Notebooks/Advanced_Deep_Learning/W_test.csv'
  weights = pd.read_csv(file_path, header=None, float_precision='round_trip')
  weights.drop(weights.columns[[0]], axis=1, inplace=True)
  weights = np.array(weights, dtype=np.float32)
  return weights

def read_bias():
  file_path = '/content/drive/My Drive/Colab Notebooks/Advanced_Deep_Learning/b_test.csv'
  bias = pd.read_csv(file_path, header=None, float_precision='round_trip')
  bias.drop(bias.columns[[0]], axis=1, inplace=True)
  bias = np.array(bias, dtype=np.float32)
  return bias


weights = read_weights()
biases = read_bias()


network_depth = 3 # excluding input layer
layer_sizes = [3,2,1]
layer_activations = ['ReLU', 'ReLU', 'Linear']

weight_vector = []
bias_vector = []
weight_vector.append(weights[0:3,:3])
weight_vector.append(weights[3:6,:2])
weight_vector.append(weights[6:8,:1])

for i in range(network_depth):
    bias_vector.append(biases[i, :layer_sizes[i]])

input = [1, 2, 1]
y_label = 0

# Create the neural network using the neuralNetwork class
primeNeuralNetwork = neuralNetwork(network_depth, layer_sizes, layer_activations)

# Set weights and biases of the neural network
primeNeuralNetwork.set_weights(weight_vector)
primeNeuralNetwork.set_bias(bias_vector)

# Forward propagation using the input
primeNeuralNetwork.forward_prop(input)
# Backword propagation using the y label for the given input
primeNeuralNetwork.backward_prop(y_label)

# Get gradients for weights and bias for each neuron
w_gradients = primeNeuralNetwork.get_w_gradients()
b_gradients = primeNeuralNetwork.get_b_gradients()

print('---------------------------- FINAL RESULT ----------------------------')
print(w_gradients)
print(b_gradients)

[array([[1., 2., 5.],
       [5., 7., 8.],
       [4., 5., 3.]], dtype=float32), array([[2., 1.],
       [5., 8.],
       [4., 2.]], dtype=float32), array([[ 1.03],
       [-1.  ]], dtype=float32)]
[array([1., 4., 7.], dtype=float32), array([5., 8.], dtype=float32), array([-0.88], dtype=float32)]
[np.float64(16.0), np.float64(25.0), np.float64(31.0)]
[np.float64(286.0), np.float64(286.0)]
[np.float64(7.699991822242737)]
last_layer_output: 7.699991822242737
[np.float64(0.999547374076985)]
---------------------------- FINAL RESULT ----------------------------
[[[np.float64(1.0595201593270445), np.float64(2.119040318654089), np.float64(1.0595201593270445)], [np.float64(-2.8487101591058064), np.float64(-5.697420318211613), np.float64(-2.8487101591058064)], [np.float64(2.119040318654089), np.float64(4.238080637308178), np.float64(2.119040318654089)]], [[np.float64(16.472540267232237), np.float64(25.73834416755037), np.float64(31.915546767762457)], [np.float64(-15.99275798523176), np.float64